<a href="https://colab.research.google.com/github/Joon-hub/MLZoomcamp/blob/main/week3_Classification_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
!curl -O https://archive.ics.uci.edu/static/public/222/bank+marketing.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  999k    0  999k    0     0   678k      0 --:--:--  0:00:01 --:--:--  678k


In [10]:
import zipfile
with zipfile.ZipFile('bank+marketing.zip','r') as zip_ref:
  file_list = zip_ref.namelist()
  for file_name in file_list:
    print(file_name)

bank.zip
bank-additional.zip


In [11]:
with zipfile.ZipFile('bank.zip','r') as zip_ref:
  zip_ref.extractall('bank')

In [14]:
df = pd.read_csv('bank/bank-full.csv',sep =';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [15]:
df.isna().sum()

,0
age,0
job,0
marital,0
education,0
default,0
balance,0
housing,0
loan,0
contact,0
day,0


**Question 1 What is the most frequent observation (mode) for the column education?**

- unknown
- primary
- secondary
- tertiary

In [16]:
df.education.value_counts()

,count
education,
secondary,23202
tertiary,13301
primary,6851
unknown,1857


In [18]:
df.education.mode()

,education
0,secondary


**Question 2**
**Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.**

What are the two features that have the biggest correlation?

- age and balance
- day and campaign
- day and pdays
- pdays and previous

In [20]:
df.age.corr(df.balance).round(3)

0.098

In [21]:
df.day.corr(df.campaign).round(3)

0.162

In [22]:
df.day.corr(df.pdays).round(3)

-0.093

In [23]:
df.pdays.corr(df.previous).round(3)

0.455

**Target encoding**


*   Now we want to encode the y variable.
*   Let's replace the values yes/no with 1/0.


In [24]:
df.y.replace({'yes':1,'no':0},inplace=True)
df.head()

<ipython-input-24-53b17d6b3215>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.y.replace({'yes':1,'no':0},inplace=True)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,0


**Split the data**

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
* Make sure that the target value `y` is not in your dataframe.

In [28]:
X = df.drop('y',axis=1)
y = df.y

In [31]:
from sklearn.model_selection import train_test_split
df_train_full, df_test, y_full_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
df_train, df_val, y_train, y_val = train_test_split(df_train_full, y_full_train, test_size=0.25, random_state=42)
len(df_train_full),len(df_train),len(df_val),len(df_test)

(36168, 27126, 9042, 9043)

**Question 3**

* Calculate the mutual information score between `y` and other categorical variables in the dataset. Use the training set only.
* Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the biggest mutual information score?
  
- `contact`
- `education`
- `housing`
- `poutcome`

In [27]:
from sklearn.metrics import mutual_info_score
print(f'mutual info score between y and contact: {mutual_info_score(df.y,df.contact).round(2)}')
print(f'mutual info score between y and education: {mutual_info_score(df.y,df.education).round(2)}')
print(f'mutual info score between y and housing: {mutual_info_score(df.y,df.housing).round(2)}')
print(f'mutual info score between y and poutcome: {mutual_info_score(df.y,df.poutcome).round(2)}')

mutual info score between y and contact: 0.01
mutual info score between y and education: 0.0
mutual info score between y and housing: 0.01
mutual info score between y and poutcome: 0.03


**Question 4**

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.6
- 0.7
- 0.8
- 0.9

In [35]:
from sklearn.linear_model import LogisticRegression

In [37]:
df_train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
20326,32,technician,single,tertiary,no,1100,yes,yes,cellular,11,aug,67,1,-1,0,unknown
24301,38,entrepreneur,married,secondary,no,0,yes,yes,cellular,17,nov,258,1,-1,0,unknown
38618,49,blue-collar,married,secondary,no,3309,yes,no,cellular,15,may,349,2,-1,0,unknown
18909,37,housemaid,married,primary,no,2410,no,no,cellular,4,aug,315,1,-1,0,unknown
23081,31,self-employed,married,tertiary,no,3220,no,no,cellular,26,aug,74,4,-1,0,unknown


In [41]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
age,27126.0,40.940574,10.596791,18.0,33.0,39.0,48.0,95.0
balance,27126.0,1382.219531,3077.844180,-8019.0,70.0,445.0,1449.0,102127.0
day,27126.0,15.837094,8.330251,1.0,8.0,16.0,21.0,31.0
duration,27126.0,255.993770,255.736080,0.0,102.0,179.0,318.0,4918.0
campaign,27126.0,2.772027,3.132857,1.0,1.0,2.0,3.0,63.0
pdays,27126.0,39.763327,99.753668,-1.0,-1.0,-1.0,-1.0,871.0
previous,27126.0,0.572071,2.497945,0.0,0.0,0.0,0.0,275.0


In [38]:
categorical = df_train.select_dtypes(include='object').columns
categorical

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object')

In [44]:
numerical = df_train.select_dtypes(exclude='object').columns
numerical

Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'], dtype='object')

In [46]:
df_train[categorical].head()

,job,marital,education,default,housing,loan,contact,month,poutcome
20326,technician,single,tertiary,no,yes,yes,cellular,aug,unknown
24301,entrepreneur,married,secondary,no,yes,yes,cellular,nov,unknown
38618,blue-collar,married,secondary,no,yes,no,cellular,may,unknown
18909,housemaid,married,primary,no,no,no,cellular,aug,unknown
23081,self-employed,married,tertiary,no,no,no,cellular,aug,unknown


In [45]:
df_train[numerical].head()

,age,balance,day,duration,campaign,pdays,previous
20326,32,1100,11,67,1,-1,0
24301,38,0,17,258,1,-1,0
38618,49,3309,15,349,2,-1,0
18909,37,2410,4,315,1,-1,0
23081,31,3220,26,74,4,-1,0


In [50]:
ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')
X_train_cat = ohe.fit_transform(df_train[categorical])
X_test_cat = ohe.transform(df_test[categorical])
X_val_cat = ohe.transform(df_val[categorical])

X_train = np.hstack((df_train[numerical],X_train_cat))
X_test = np.hstack((df_test[numerical],X_test_cat))
X_val = np.hstack((df_val[numerical],X_val_cat))


In [67]:
ohe.get_feature_names_out()

array(['job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired',
       'job_self-employed', 'job_services', 'job_student',
       'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'education_unknown', 'default_no', 'default_yes', 'housing_no',
       'housing_yes', 'loan_no', 'loan_yes', 'contact_cellular',
       'contact_telephone', 'contact_unknown', 'month_apr', 'month_aug',
       'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'poutcome_failure', 'poutcome_other', 'poutcome_success',
       'poutcome_unknown'], dtype=object)

**Question 4**

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.6
- 0.7
- 0.8
- 0.9

In [87]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train,y_train)

y_pred_val = model.predict(X_val)
y_pred = model.predict(X_test)

print(f'model accuracy on validation set: {round(accuracy_score(y_val,y_pred_val),3)}')
print(f'model accuracy on test set: {round(accuracy_score(y_test, y_pred),3)}')


model accuracy on validation set: 0.9
model accuracy on test set: 0.899


**Question 5**

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

- `age`
- `balance`
- `marital`
- `previous`

> **Note**: The difference doesn't have to be positive.

In [79]:
#age

ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')
X_train_cat = ohe.fit_transform(df_train[categorical])
X_test_cat = ohe.transform(df_test[categorical])
X_val_cat = ohe.transform(df_val[categorical])

X_train = np.hstack((df_train[numerical].drop('age', axis = 1),X_train_cat))
X_test = np.hstack((df_test[numerical].drop('age', axis = 1),X_test_cat))
X_val = np.hstack((df_val[numerical].drop('age', axis = 1),X_val_cat))

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train,y_train)

y_pred_val = model.predict(X_val)
y_pred = model.predict(X_test)

print(f'model accuracy on validation set after eliminating age column: {round(accuracy_score(y_val,y_pred_val),3)}')
print(f'model accuracy on test set after eliminating age column: {round(accuracy_score(y_test, y_pred),3)}')

model accuracy on validation set after eliminating age column: 0.901
model accuracy on test set after eliminating age column: 0.899


In [80]:
# balance

ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')
X_train_cat = ohe.fit_transform(df_train[categorical])
X_test_cat = ohe.transform(df_test[categorical])
X_val_cat = ohe.transform(df_val[categorical])

X_train = np.hstack((df_train[numerical].drop('balance', axis = 1),X_train_cat))
X_test = np.hstack((df_test[numerical].drop('balance', axis = 1),X_test_cat))
X_val = np.hstack((df_val[numerical].drop('balance', axis = 1),X_val_cat))

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train,y_train)

y_pred_val = model.predict(X_val)
y_pred = model.predict(X_test)

print(f'model accuracy on validation set after eliminating balance column: {round(accuracy_score(y_val,y_pred_val),3)}')
print(f'model accuracy on test set after eliminating balance column: {round(accuracy_score(y_test, y_pred),3)}')

model accuracy on validation set after eliminating balance column: 0.9
model accuracy on test set after eliminating balance column: 0.899


In [84]:
# marital

ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')
X_train_cat = ohe.fit_transform(df_train[categorical].drop('marital', axis = 1))
X_test_cat = ohe.transform(df_test[categorical].drop('marital', axis = 1))
X_val_cat = ohe.transform(df_val[categorical].drop('marital', axis = 1))

X_train = np.hstack((df_train[numerical],X_train_cat))
X_test = np.hstack((df_test[numerical],X_test_cat))
X_val = np.hstack((df_val[numerical],X_val_cat))

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train,y_train)

y_pred_val = model.predict(X_val)
y_pred = model.predict(X_test)

print(f'model accuracy on validation set after eliminating marital column: {round(accuracy_score(y_val,y_pred_val),3)}')
print(f'model accuracy on test set after eliminating marital column: {round(accuracy_score(y_test, y_pred),3)}')

model accuracy on validation set after eliminating marital column: 0.901
model accuracy on test set after eliminating marital column: 0.899


In [86]:
# previous

ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')
X_train_cat = ohe.fit_transform(df_train[categorical])
X_test_cat = ohe.transform(df_test[categorical])
X_val_cat = ohe.transform(df_val[categorical])

X_train = np.hstack((df_train[numerical].drop('previous', axis = 1),X_train_cat))
X_test = np.hstack((df_test[numerical].drop('previous', axis = 1),X_test_cat))
X_val = np.hstack((df_val[numerical].drop('previous', axis = 1),X_val_cat))

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train,y_train)

y_pred_val = model.predict(X_val)
y_pred = model.predict(X_test)

print(f'model accuracy on validation set after eliminating previous column: {round(accuracy_score(y_val,y_pred_val),3)}')
print(f'model accuracy on test set after eliminating previous column: {round(accuracy_score(y_test, y_pred),3)}')

model accuracy on validation set after eliminating previous column: 0.9
model accuracy on test set after eliminating previous column: 0.899


In [89]:
# Define the features to eliminate one by one
features_to_check = ['age', 'balance', 'marital', 'previous']

# Initialize the OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Encode the categorical data once for all features
X_train_cat = ohe.fit_transform(df_train[categorical])
X_test_cat = ohe.transform(df_test[categorical])
X_val_cat = ohe.transform(df_val[categorical])

# Combine numerical and categorical data (before eliminating features)
X_train_all = np.hstack((df_train[numerical], X_train_cat))
X_test_all = np.hstack((df_test[numerical], X_test_cat))
X_val_all = np.hstack((df_val[numerical], X_val_cat))

# Train the model with all features
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_all, y_train)

# Calculate the accuracy with all features
y_pred_all = model.predict(X_test_all)
accuracy_all = accuracy_score(y_test, y_pred_all)
print(f"Accuracy with all features: {accuracy_all}")

# Dictionary to store accuracy differences
accuracy_diff = {}

# Loop through each feature and eliminate one at a time
for feature in features_to_check:

    # Drop the feature from numerical columns if it is numerical
    if feature in numerical:
        X_train_new = np.hstack((df_train[numerical].drop(columns=[feature]), X_train_cat))
        X_test_new = np.hstack((df_test[numerical].drop(columns=[feature]), X_test_cat))
    # Drop the feature from categorical columns if it is categorical
    elif feature in categorical:
        X_train_cat_new = ohe.fit_transform(df_train[categorical].drop(columns=[feature]))
        X_test_cat_new = ohe.transform(df_test[categorical].drop(columns=[feature]))

        # Combine updated categorical data with numerical data
        X_train_new = np.hstack((df_train[numerical], X_train_cat_new))
        X_test_new = np.hstack((df_test[numerical], X_test_cat_new))

    # Train the model without the feature
    model.fit(X_train_new, y_train)
    y_pred = model.predict(X_test_new)

    # Calculate accuracy without the feature
    accuracy_without_feature = accuracy_score(y_test, y_pred)

    # Store the difference in accuracy
    accuracy_diff[feature] = accuracy_all - accuracy_without_feature
    print(f"Accuracy without {feature}: {accuracy_without_feature}")
    print(f"Difference: {accuracy_diff[feature]}")

# Find the feature with the smallest difference
least_important_feature = min(accuracy_diff, key=accuracy_diff.get)
print(f"The least important feature is: {least_important_feature}")


Accuracy with all features: 0.8988167643481145
Accuracy without age: 0.8993696782041358
Difference: -0.0005529138560212976
Accuracy without balance: 0.8994802609753401
Difference: -0.0006634966272255349
Accuracy without marital: 0.8993696782041358
Difference: -0.0005529138560212976
Accuracy without previous: 0.8988167643481145
Difference: 0.0
The least important feature is: balance


**Question 6**

* Now let's train a regularized logistic regression.
* Let's try the following values of the parameter `C`: `[0, 0.01, 0.1, 1, 10]`.
* Train models using all the features as in Q4.
* Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these `C` leads to the best accuracy on the validation set?

- 0.01
- 0.1
- 1
- 10
- 100

In [93]:
c = [0.0001, 0.01, 0.1, 1, 10,100]
for i in c:
  model = LogisticRegression(solver='liblinear', C=i, max_iter=1000, random_state=42)
  model.fit(X_train,y_train)
  y_pred_val = model.predict(X_val)
  y_pred = model.predict(X_test)

  print(f'model accuracy on validation set for c = {i}: {round(accuracy_score(y_val,y_pred_val),7)}')

model accuracy on validation set for c = 0.0001: 0.8886308
model accuracy on validation set for c = 0.01: 0.8983632
model accuracy on validation set for c = 0.1: 0.9005751
model accuracy on validation set for c = 1: 0.9003539
model accuracy on validation set for c = 10: 0.9001327
model accuracy on validation set for c = 100: 0.9002433
